In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None) # Show all columns in a wide DataFrame

data = pd.read_excel('./nba_player_data.xlsx')

In [2]:
data.sample(10)

,Year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
4673,2010-11,Regular%20Season,2584,154,Willie Green,1610612740,NOH,77,1674,270,609,0.443,54,155,0.348,78,100,0.780,22,139,161,74,36,13,67,143,672,528,1.10,0.54
7410,2014-15,Regular%20Season,203935,204,Marcus Smart,1610612738,BOS,67,1808,175,477,0.367,91,272,0.335,82,127,0.646,61,161,222,208,99,18,90,176,523,633,2.31,1.10
8126,2015-16,Regular%20Season,200757,221,Thabo Sefolosha,1610612737,ATL,75,1758,188,371,0.507,42,123,0.341,62,99,0.626,52,282,334,107,85,37,68,109,480,755,1.57,1.25
13146,2022-23,Regular%20Season,203897,10,Zach LaVine,1610612741,CHI,77,2768,673,1388,0.485,204,544,0.375,363,428,0.848,42,303,345,327,69,18,194,159,1913,1698,1.69,0.36
12422,2021-22,Regular%20Season,1629611,107,Terance Mann,1610612746,LAC,81,2317,334,690,0.484,76,208,0.365,128,164,0.780,104,321,425,209,55,21,84,178,872,1106,2.49,0.66
5152,2010-11,Playoffs,2124,182,Malik Allen,1610612753,ORL,1,7,0,1,0.000,0,0,0.000,1,2,0.500,0,1,1,0,0,0,0,2,1,0,0.00,0.00
825,2004-05,Regular%20Season,2744,195,Al Jefferson,1610612738,BOS,71,1050,195,369,0.528,0,3,0.000,85,135,0.630,119,193,312,24,22,55,66,195,475,598,0.36,0.33
8050,2015-16,Regular%20Season,201158,145,Marco Belinelli,1610612758,SAC,68,1672,245,635,0.386,91,297,0.306,115,138,0.833,10,107,117,127,37,2,80,91,696,486,1.59,0.46
836,2004-05,Regular%20Season,2585,205,Zaza Pachulia,1610612749,MIL,74,1396,160,354,0.452,0,1,0.000,138,185,0.746,131,247,378,60,44,34,70,166,458,663,0.86,0.63
9610,2017-18,Regular%20Season,203487,313,Michael Carter-Williams,1610612766,CHA,52,835,76,229,0.332,14,59,0.237,73,89,0.820,37,101,138,116,44,23,52,99,239,339,2.23,0.85


In [3]:
data.shape

(13893, 30)

## Data cleaning  & analysis preparation

In [4]:
data.isna().sum()

Year           0
Season_type    0
PLAYER_ID      0
RANK           0
PLAYER         0
TEAM_ID        0
TEAM           0
GP             0
MIN            0
FGM            0
FGA            0
FG_PCT         0
FG3M           0
FG3A           0
FG3_PCT        0
FTM            0
FTA            0
FT_PCT         0
OREB           0
DREB           0
REB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
EFF            0
AST_TOV        0
STL_TOV        0
dtype: int64

In [5]:
# Drop columns we don't need
data.drop(columns=['RANK', 'EFF'], inplace=True)

In [6]:
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [7]:
data.TEAM.unique()

array(['MIN', 'SAC', 'ORL', 'BOS', 'MIL', 'DEN', 'DAL', 'CLE', 'TOR',
       'NYK', 'POR', 'IND', 'LAL', 'SAS', 'NOH', 'NJN', 'LAC', 'PHX',
       'GSW', 'ATL', 'HOU', 'SEA', 'MIA', 'CHI', 'MEM', 'DET', 'UTA',
       'PHI', 'WAS', 'CHA', 'NOK', 'OKC', 'BKN', 'NOP'], dtype=object)

In [8]:
data.TEAM.nunique() # we should have 30 franchises so we have to standardize

34

In [9]:
data.TEAM.value_counts()


TEAM
BOS    550
SAS    539
MIA    539
DAL    524
DEN    505
MEM    500
ATL    496
HOU    492
LAL    491
MIL    490
LAC    480
IND    475
CLE    472
PHI    471
UTA    469
CHI    465
POR    461
TOR    460
GSW    452
WAS    449
ORL    418
DET    415
PHX    413
NYK    401
OKC    383
SAC    368
MIN    364
CHA    344
BKN    305
NOP    210
NJN    195
NOH    177
SEA     88
NOK     32
Name: count, dtype: int64

In [10]:
# Franchises that need to be standardized 'NOH' => 'NOP' || 'NJN' => 'BKN' || 'SEA' => 'OKC' || 'NOK' => 'NOP'
team_mapping = {'NOH': 'NOP', 'NJN': 'BKN', 'SEA': 'OKC', 'NOK': 'NOP'}
# data['TEAM'].replace(to_replace=team_mapping, value='NO', inplace=True)

data['TEAM'] = data['TEAM'].map(team_mapping).fillna(data['TEAM'])
data

,Year,Season_type,PLAYER_ID,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV,season_start_year
0,2003-04,Regular%20Season,708,Kevin Garnett,1610612750,MIN,82,3232,804,1611,0.499,11,43,0.256,368,465,0.791,245,894,1139,409,120,178,212,202,1987,1.93,0.57,2003
1,2003-04,Regular%20Season,978,Peja Stojakovic,1610612758,SAC,81,3265,665,1386,0.480,240,554,0.433,394,425,0.927,91,417,508,173,108,14,153,159,1964,1.13,0.71,2003
2,2003-04,Regular%20Season,1503,Tracy McGrady,1610612753,ORL,67,2675,653,1566,0.417,174,513,0.339,398,500,0.796,95,307,402,370,93,42,179,129,1878,2.07,0.52,2003
3,2003-04,Regular%20Season,1718,Paul Pierce,1610612738,BOS,80,3101,602,1497,0.402,115,384,0.299,517,631,0.819,69,453,522,410,131,52,303,234,1836,1.35,0.43,2003
4,2003-04,Regular%20Season,2072,Michael Redd,1610612749,MIL,82,3021,633,1439,0.440,127,363,0.350,383,441,0.868,118,289,407,185,81,6,116,152,1776,1.59,0.70,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13888,2022-23,Playoffs,203648,Thanasis Antetokounmpo,1610612749,MIL,2,5,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022
13889,2022-23,Playoffs,1628418,Thomas Bryant,1610612743,DEN,1,1,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022
13890,2022-23,Playoffs,2617,Udonis Haslem,1610612748,MIA,2,3,0,3,0.000,0,1,0.000,0,0,0.000,0,1,1,0,0,0,0,0,0,0.00,0.00,2022
13891,2022-23,Playoffs,1628427,Vlatko Cancar,1610612743,DEN,5,10,0,5,0.000,0,4,0.000,0,0,0.000,0,3,3,1,0,0,0,0,0,0.00,0.00,2022


In [11]:
data.TEAM.unique()

array(['MIN', 'SAC', 'ORL', 'BOS', 'MIL', 'DEN', 'DAL', 'CLE', 'TOR',
       'NYK', 'POR', 'IND', 'LAL', 'SAS', 'NOP', 'BKN', 'LAC', 'PHX',
       'GSW', 'ATL', 'HOU', 'OKC', 'MIA', 'CHI', 'MEM', 'DET', 'UTA',
       'PHI', 'WAS', 'CHA'], dtype=object)

In [12]:
data['Season_type'].replace('Regular%20Season', 'RS', inplace=True)
data

,Year,Season_type,PLAYER_ID,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV,season_start_year
0,2003-04,RS,708,Kevin Garnett,1610612750,MIN,82,3232,804,1611,0.499,11,43,0.256,368,465,0.791,245,894,1139,409,120,178,212,202,1987,1.93,0.57,2003
1,2003-04,RS,978,Peja Stojakovic,1610612758,SAC,81,3265,665,1386,0.480,240,554,0.433,394,425,0.927,91,417,508,173,108,14,153,159,1964,1.13,0.71,2003
2,2003-04,RS,1503,Tracy McGrady,1610612753,ORL,67,2675,653,1566,0.417,174,513,0.339,398,500,0.796,95,307,402,370,93,42,179,129,1878,2.07,0.52,2003
3,2003-04,RS,1718,Paul Pierce,1610612738,BOS,80,3101,602,1497,0.402,115,384,0.299,517,631,0.819,69,453,522,410,131,52,303,234,1836,1.35,0.43,2003
4,2003-04,RS,2072,Michael Redd,1610612749,MIL,82,3021,633,1439,0.440,127,363,0.350,383,441,0.868,118,289,407,185,81,6,116,152,1776,1.59,0.70,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13888,2022-23,Playoffs,203648,Thanasis Antetokounmpo,1610612749,MIL,2,5,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022
13889,2022-23,Playoffs,1628418,Thomas Bryant,1610612743,DEN,1,1,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022
13890,2022-23,Playoffs,2617,Udonis Haslem,1610612748,MIA,2,3,0,3,0.000,0,1,0.000,0,0,0.000,0,1,1,0,0,0,0,0,0,0.00,0.00,2022
13891,2022-23,Playoffs,1628427,Vlatko Cancar,1610612743,DEN,5,10,0,5,0.000,0,4,0.000,0,0,0.000,0,3,3,1,0,0,0,0,0,0.00,0.00,2022


In [13]:
rs_df = data[data['Season_type']=='RS']
playoffs_df = data[data['Season_type']=='Playoffs']
rs_df

,Year,Season_type,PLAYER_ID,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV,season_start_year
0,2003-04,RS,708,Kevin Garnett,1610612750,MIN,82,3232,804,1611,0.499,11,43,0.256,368,465,0.791,245,894,1139,409,120,178,212,202,1987,1.93,0.57,2003
1,2003-04,RS,978,Peja Stojakovic,1610612758,SAC,81,3265,665,1386,0.480,240,554,0.433,394,425,0.927,91,417,508,173,108,14,153,159,1964,1.13,0.71,2003
2,2003-04,RS,1503,Tracy McGrady,1610612753,ORL,67,2675,653,1566,0.417,174,513,0.339,398,500,0.796,95,307,402,370,93,42,179,129,1878,2.07,0.52,2003
3,2003-04,RS,1718,Paul Pierce,1610612738,BOS,80,3101,602,1497,0.402,115,384,0.299,517,631,0.819,69,453,522,410,131,52,303,234,1836,1.35,0.43,2003
4,2003-04,RS,2072,Michael Redd,1610612749,MIL,82,3021,633,1439,0.440,127,363,0.350,383,441,0.868,118,289,407,185,81,6,116,152,1776,1.59,0.70,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13671,2022-23,RS,1631214,Alondes Williams,1610612751,BKN,1,5,0,0,0.000,0,0,0.000,0,0,0.000,0,1,1,0,0,0,2,1,0,0.00,0.00,2022
13672,2022-23,RS,1629126,Deonte Burton,1610612758,SAC,2,7,0,2,0.000,0,1,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022
13673,2022-23,RS,1628402,Frank Jackson,1610612762,UTA,1,5,0,3,0.000,0,1,0.000,0,0,0.000,1,1,2,1,0,0,0,0,0,0.00,0.00,2022
13674,2022-23,RS,1630701,Michael Foster Jr.,1610612755,PHI,1,1,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.00,0.00,2022


In [14]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'AST_TOV', 'STL_TOV', 'season_start_year'],
      dtype='object')

In [15]:
total_cols = ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
              'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

## Which player stats are correlated with each other?

In [16]:
data_per_min = data.groupby(['PLAYER','PLAYER_ID','Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
  data_per_min[col] = data_per_min[col]/data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']

data_per_min = data_per_min[data_per_min['MIN']>=50]

fig = px.imshow(data_per_min.corr(numeric_only=True))
fig.show()

## How are minutes played distributed?

In [17]:
fig = px.histogram(x=playoffs_df['MIN'], histnorm='percent')
fig.show()

In [18]:
def hist_data(df=rs_df, min_MIN=0, min_GP=0):
  return df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'MIN']/\
  df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'GP']

In [19]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=hist_data(rs_df, 50, 5), histnorm='percent', name='Regular Season',
                           xbins={'start': 0, 'end': 38, 'size': 1}))
fig.add_trace(go.Histogram(x=hist_data(playoffs_df, 5, 1), histnorm='percent', name='Playoffs',
                           xbins={'start': 0, 'end': 38, 'size': 1}))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.40)
fig.show()


In [20]:
((hist_data(rs_df, 50, 5)>=7)&(hist_data(rs_df, 50, 5)<=34)).mean()

0.8508713876020295

In [21]:
((hist_data(playoffs_df, 50, 5)>=7)&(hist_data(playoffs_df, 50, 5)<=34)).mean()

0.6966717095310136

## How has the game changed over the past 20 years ?

In [29]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
change_df['POSS_est'] = change_df['FGA']-change_df['OREB']+change_df['TOV']+0.44*change_df['FTA']
change_df = change_df[list(change_df.columns[0:2])+['POSS_est']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df

,season_start_year,MIN,POSS_est,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,612550,235120.12,88563,202419,13152,38056,46256,61698,30630,76689,107319,53876,20146,12844,36184,54818,236534,0.437523,0.345596,0.749716,0.608335,0.188006,1.168537,0.148504,0.304803,0.510375,1.488945
1,2004,634977,245843.32,94392,210915,14808,41639,51827,68603,31534,78376,109910,55502,19633,12888,36277,59611,255419,0.447536,0.355628,0.755463,0.587995,0.197421,1.211004,0.156878,0.325264,0.524471,1.529950
2,2005,638826,245555.12,94488,208086,15157,42381,51915,69573,29336,78555,107891,53991,18764,12319,36193,60255,256048,0.454081,0.357637,0.746195,0.571406,0.203671,1.230491,0.160412,0.334347,0.530927,1.491753
3,2006,633842,247123.48,95270,208309,15938,44565,51311,68267,29115,78389,107504,55448,18926,12029,37892,58169,257789,0.457349,0.357635,0.751622,0.582009,0.213937,1.237532,0.167293,0.327720,0.535419,1.463317
4,2007,635553,248757.96,97602,213855,17179,47561,49743,65859,29456,80808,110264,56909,19026,12495,35381,55606,262126,0.456393,0.361199,0.755295,0.583072,0.222398,1.225718,0.176011,0.307961,0.534650,1.608462
5,2008,635960,247268.28,97242,212197,17520,47859,50310,65337,28841,79513,108354,54834,19086,12626,35164,55626,262314,0.458263,0.366075,0.770008,0.563892,0.225540,1.236181,0.180169,0.307907,0.539226,1.559379
6,2009,633931,248970.56,98508,213747,16911,47740,49164,64824,28692,80562,109254,55446,18852,12761,35393,55129,263091,0.460863,0.354231,0.758423,0.562858,0.223348,1.230852,0.171671,0.303274,0.537934,1.566581
7,2010,634350,247418.12,97126,212331,16875,47215,48998,64123,28618,79787,108405,55872,19164,12840,35491,54434,260125,0.457427,0.357408,0.764125,0.575253,0.222365,1.225092,0.173743,0.301995,0.535701,1.574258
8,2011,519631,201241.24,78019,174465,13687,39378,36460,48446,24302,66191,90493,44620,16425,11010,29762,42301,206185,0.447190,0.347580,0.752591,0.571912,0.225707,1.181813,0.175432,0.277683,0.522049,1.499227
9,2012,635884,248201.92,97235,215105,18808,52569,44125,58618,29237,81362,110599,57694,20376,13444,36542,52548,257403,0.452035,0.357777,0.752755,0.593346,0.244388,1.196639,0.193428,0.272509,0.529748,1.578841


In [34]:
change_per48_df = change_df.copy()
for col in change_per48_df.columns[2:18]:
  change_per48_df[col] = (change_per48_df[col]/change_per48_df['MIN'])*48*5 # avg stats per team. 48minutes, 5 players in the court
change_per48_df.drop(columns='MIN', inplace=True)

fig = go.Figure()
for col in change_per48_df.columns[1:]:
  fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'],
                           y=change_per48_df[col], name=col))
fig.show()

In [37]:
change_per100_df = change_df.copy()
for col in change_per100_df.columns[3:18]:
  change_per100_df[col] = (change_per100_df[col]/change_per100_df['POSS_est'])*100
change_per100_df.drop(columns=['MIN', 'POSS_est'], inplace=True)
change_per100_df

fig = go.Figure()
for col in change_per100_df.columns[1:]:
  fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'],
                           y=change_per100_df[col], name=col))
fig.show()

## Compare RS to Playoffs

In [40]:
rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df,playoffs_change_df]:
  i['POSS_est'] = i['FGA']-i['OREB']+i['TOV']+0.44*i['FTA']
  i['POSS_per_48'] = (i['POSS_est']/i['MIN'])*48*5
  
  i['FG%'] = i['FGM']/i['FGA']
  i['3PT%'] = i['FG3M']/i['FG3A']
  i['FT%'] = i['FTM']/i['FTA']
  i['AST%'] = i['AST']/i['FGM']
  i['FG3A%'] = i['FG3A']/i['FGA']
  i['PTS/FGA'] = i['PTS']/i['FGA']
  i['FG3M/FGM'] = i['FG3M']/i['FGM']
  i['FTA/FGA'] = i['FTA']/i['FGA']
  i['TRU%'] = 0.5*i['PTS']/(i['FGA']+0.475*i['FTA'])
  i['AST_TOV'] = i['AST']/i['TOV']
  for col in total_cols:
    i[col] = 100*i[col]/i['POSS_est']
  i.drop(columns=['MIN','POSS_est'], inplace=True)
    
rs_change_df
  

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,37.800947,86.178386,5.594271,16.114916,19.645559,26.136567,13.055116,32.513155,45.568271,23.000941,8.565079,5.458512,15.376641,23.158949,100.841724,92.266157,0.438636,0.347149,0.751650,0.608475,0.186995,1.170151,0.147993,0.303284,0.511403,1.495837
1,2004,38.388612,85.787163,5.980437,16.820059,21.036794,27.822899,12.824709,31.873378,44.698087,22.721057,8.025861,5.193000,14.795471,24.166138,103.794454,93.029661,0.447487,0.355554,0.756096,0.591870,0.196067,1.209907,0.155787,0.324325,0.524198,1.535677
2,2005,38.472213,84.791507,6.146582,17.154664,21.042641,28.232563,11.996003,31.969123,43.965126,22.128308,7.693919,5.044334,14.782169,24.430112,104.133649,92.351525,0.453727,0.358304,0.745332,0.575176,0.202316,1.228114,0.159767,0.332965,0.530201,1.496959
3,2006,38.624284,84.277535,6.415603,17.911765,20.740458,27.572994,11.760076,31.649906,43.409982,22.507908,7.653935,4.869514,15.350424,23.496941,104.404630,93.722146,0.458299,0.358178,0.752202,0.582740,0.212533,1.238819,0.166103,0.327169,0.536097,1.466273
4,2007,39.302651,85.963857,6.913089,19.098030,19.871915,26.309097,11.812361,32.464592,44.276954,22.940445,7.673254,4.995311,14.272502,22.169990,105.390306,94.218833,0.457200,0.361979,0.755325,0.583687,0.222163,1.225984,0.175894,0.306048,0.535190,1.607318
5,2008,39.405324,85.902829,7.056794,19.240034,20.242969,26.264030,11.719750,32.120237,43.839987,22.270416,7.717505,5.099690,14.260748,22.339465,106.110410,93.520629,0.458720,0.366777,0.770749,0.565163,0.223974,1.235238,0.179082,0.305741,0.539298,1.561658
6,2009,39.635678,85.908933,6.762814,19.072827,19.584034,25.807003,11.521403,32.350117,43.871520,22.336690,7.590747,5.106518,14.257389,21.931058,105.618205,94.446220,0.461369,0.354578,0.758865,0.563550,0.222012,1.229421,0.170624,0.300400,0.537950,1.566675
7,2010,39.365965,85.839148,6.825370,19.038942,19.660641,25.751745,11.533441,32.211485,43.744926,22.722734,7.743526,5.141154,14.363525,21.891797,105.217941,93.871650,0.458602,0.358495,0.763468,0.577218,0.221798,1.225757,0.173383,0.300000,0.536436,1.581975
8,2011,38.847626,86.726418,6.827839,19.577658,18.001548,23.922451,12.109692,32.813758,44.923451,22.341500,8.176409,5.425478,14.857396,20.841236,102.524638,93.132725,0.447933,0.348757,0.752496,0.575106,0.225740,1.182162,0.175760,0.275838,0.522607,1.503729
9,2012,39.270953,86.735365,7.573088,21.109396,17.662937,23.460955,11.812003,32.747162,44.559165,23.410189,8.250677,5.430183,14.753819,20.983773,103.777931,93.838988,0.452767,0.358754,0.752865,0.596120,0.243377,1.196489,0.192842,0.270489,0.530132,1.586721


In [41]:
playoffs_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,35.686054,84.808993,5.585819,17.234704,20.084748,27.787935,12.616825,34.153483,46.770308,21.630763,8.617352,5.525322,15.581140,25.623514,97.042675,90.026989,0.420781,0.324103,0.722787,0.606141,0.203218,1.144250,0.156527,0.327653,0.495074,1.388266
1,2004,38.493052,85.871105,6.662135,18.681117,21.744019,29.129877,12.859019,31.986001,44.845020,20.419346,7.392320,5.977182,14.170768,25.459565,105.392258,91.325698,0.448265,0.356624,0.746451,0.530468,0.217548,1.227331,0.173074,0.339228,0.528506,1.440948
2,2005,38.579079,84.035511,6.535621,18.722021,22.529889,29.736769,11.258842,32.293655,43.552497,20.015720,6.907864,4.631686,14.139153,26.050947,106.223668,90.888075,0.459081,0.349087,0.757644,0.518823,0.222787,1.264033,0.169408,0.353860,0.541071,1.415624
3,2006,37.382738,84.549942,6.992852,19.990437,21.130576,28.455105,12.126933,32.856732,44.983665,21.303324,7.732215,4.836953,15.056744,24.205496,102.888905,91.216766,0.442138,0.349810,0.742593,0.569871,0.236433,1.216901,0.187061,0.336548,0.524589,1.414869
4,2007,38.229965,86.048028,6.798013,19.440385,21.869628,28.970491,12.275086,32.785111,45.060196,21.927620,7.274841,5.438411,13.480042,25.110766,105.127571,89.894953,0.444286,0.349685,0.754893,0.573572,0.225925,1.221731,0.177819,0.336678,0.526644,1.626673
5,2008,38.152032,84.530033,7.512066,21.069800,21.886609,28.800539,10.830752,32.698066,43.528819,20.767517,7.737170,5.203134,13.628482,24.832265,105.702738,90.349058,0.451343,0.356532,0.759937,0.544336,0.249258,1.250476,0.196898,0.340714,0.538145,1.523832
6,2009,38.482339,84.970176,7.252902,20.766343,22.284857,29.617681,11.568678,32.481545,44.050223,21.232554,7.279542,5.421361,13.566723,25.441768,106.502436,91.415896,0.452892,0.349262,0.752417,0.551748,0.244396,1.253410,0.188474,0.348566,0.537682,1.565047
7,2010,37.508078,85.494150,6.742183,19.783432,22.074002,28.536680,12.093662,32.824681,44.918344,20.349257,7.778393,5.958208,14.043374,23.730574,103.832341,89.617188,0.438721,0.340799,0.773531,0.542530,0.231401,1.214496,0.179753,0.333785,0.524146,1.449029
8,2011,37.814786,86.307149,6.479555,19.445183,19.523407,25.905182,11.668414,33.831881,45.500295,20.123125,7.985367,6.023248,13.962984,23.186898,101.632535,90.747933,0.438142,0.333222,0.753649,0.532150,0.225302,1.177568,0.171350,0.300151,0.515315,1.441176
9,2012,37.771884,85.632344,7.645745,22.220248,19.472856,25.919393,11.300475,33.273267,44.573742,20.805310,7.601330,5.215604,14.263597,23.939748,102.662369,91.369052,0.441094,0.344089,0.751285,0.550815,0.259484,1.198874,0.202419,0.302682,0.524087,1.458630


In [47]:
# percent change through RS to Playoffs
comp_change_df = round(100*(playoffs_change_df-rs_change_df)/rs_change_df, 3)
comp_change_df['season_start_year'] = list(range(2003, 2023))
comp_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,-5.595,-1.589,-0.151,6.949,2.236,6.318,-3.357,5.045,2.638,-5.957,0.610,1.224,1.330,10.642,-3.767,-2.427,-4.070,-6.639,-3.840,-0.384,8.676,-2.213,5.766,8.035,-3.193,-7.191
1,2004,0.272,0.098,11.399,11.065,3.362,4.697,0.268,0.353,0.329,-10.130,-7.894,15.101,-4.222,5.352,1.539,-1.832,0.174,0.301,-1.276,-10.374,10.956,1.440,11.097,4.595,0.822,-6.168
2,2005,0.278,-0.892,6.329,9.137,7.068,5.328,-6.145,1.015,-0.939,-9.547,-10.217,-8.180,-4.350,6.635,2.007,-1.585,1.180,-2.572,1.652,-9.798,10.118,2.925,6.035,6.275,2.050,-5.433
3,2006,-3.214,0.323,8.998,11.605,1.881,3.199,3.120,3.813,3.625,-5.352,1.023,-0.669,-1.913,3.016,-1.452,-2.673,-3.526,-2.336,-1.277,-2.208,11.245,-1.769,12.618,2.867,-2.147,-3.506
4,2007,-2.729,0.098,-1.665,1.793,10.053,10.116,3.917,0.987,1.769,-4.415,-5.192,8.870,-5.552,13.265,-0.249,-4.589,-2.824,-3.396,-0.057,-1.733,1.693,-0.347,1.095,10.008,-1.597,1.204
5,2008,-3.181,-1.598,6.452,9.510,8.120,9.658,-7.585,1.799,-0.710,-6.748,0.255,2.028,-4.434,11.159,-0.384,-3.391,-1.608,-2.793,-1.403,-3.685,11.289,1.234,9.948,11.439,-0.214,-2.422
6,2009,-2.910,-1.093,7.247,8.879,13.791,14.766,0.410,0.406,0.407,-4.943,-4.100,6.166,-4.844,16.008,0.837,-3.209,-1.837,-1.499,-0.850,-2.094,10.082,1.951,10.461,16.034,-0.050,-0.104
7,2010,-4.720,-0.402,-1.219,3.910,12.275,10.815,4.857,1.904,2.682,-10.445,0.450,15.892,-2.229,8.399,-1.317,-4.532,-4.335,-4.936,1.318,-6.009,4.330,-0.919,3.674,11.262,-2.291,-8.404
8,2011,-2.659,-0.483,-5.101,-0.677,8.454,8.288,-3.644,3.103,1.284,-9.929,-2.337,11.018,-6.020,11.255,-0.870,-2.561,-2.186,-4.454,0.153,-7.469,-0.194,-0.389,-2.509,8.814,-1.395,-4.160
9,2012,-3.817,-1.272,0.959,5.262,10.247,10.479,-4.331,1.607,0.033,-11.127,-7.870,-3.952,-3.323,14.087,-1.075,-2.632,-2.578,-4.088,-0.210,-7.600,6.618,0.199,4.966,11.902,-1.140,-8.073


In [48]:
fig = go.Figure()
for col in comp_change_df.columns[1:]:
  fig.add_trace(go.Scatter(x=comp_change_df['season_start_year'],
                           y=comp_change_df [col], name=col))
fig.show()